In [17]:
import numpy as np 
import scipy
import matplotlib.pyplot as plt 

def GetPWR(signal):
    Pwr = np.sum(np.abs(signal**2))
    return Pwr

def Normalize(X):
    for i in range(X.shape[0]):
        x = X[i,0] + 1j* X[i,1]
        x /= np.sqrt(GetPWR(x))
        X[i,0] = np.real(x)
        X[i,1] = np.imag(x)

def AugL(Y):
    return np.vstack((Y,Y))


def TPD(X):
    Xc = X[:,0,:] + 1j* X[:,1,:]
    diff1 = np.gradient(np.unwrap(np.angle(Xc)))[1]
    out = np.dstack([diff1/(2*np.pi)])
    return np.swapaxes(out,1,2)

def Transient(X):
    return X[:,:,:85]
    
def Preamble(X):
    return X[:,:,:137]

def FilterNan(X):
    Idx = [] 
    for i in range(X.shape[0]):
        arr = X_train[i,0]

        # Check if any element is NaN
        if np.isnan(arr).any():
            print(f"Index {i} is bad")
            Idx.append(i)

    X = np.delete(X, Idx, axis=0)

def Mbed(X):
    Xc = X[:,0,:] + 1j* X[:,1,:]
    Xm = np.abs(Xc)
    Xp = np.angle(Xc)
    Xf = [] 
    for i in range(Xc.shape[0]):
        Xfft = np.fft.fft(Xc[0])
        Xfft2 = np.abs(Xfft)**2
        Xf.append(Xfft2)
        
    PSD = np.vstack(Xf)
    Out = np.dstack((Xm, Xp, PSD))
    Out = np.swapaxes(Out, 1,2)
    return Out 

def TP(X):
    return X[:,:,:137]

In [2]:
# Download the dataset from 
# Change Directory to where the data is

# For user to change 
Your_Dir = "..."

X_train = np.load(f"{Your_Dir}/X_train.npy")
Y_train = np.load(f"{Your_Dir}/Y_train.npy")
X_test = np.load(f"{Your_Dir}/X_test.npy")
Y_test = np.load(f"{Your_Dir}/Y_test.npy")

In [19]:
# Samples the first 137 components 

X_train = Preamble(X_train)
X_test = Preamble(X_test)

In [20]:
# Select a Technique [Mbed, TPD , TP] and comment out the others 

# X_train = Mbed(X_train)
# X_test = Mbed(X_test)

X_train = TPD(X_train)
X_test = TPD(X_test)

# X_train = TP(X_train)
# X_test = TP(X_test)

In [21]:
import keras.layers as l
import tensorflow as tf
from keras.layers import Conv1D, Conv2D, MaxPooling2D,MaxPooling1D, Dense, Flatten, Dropout, BatchNormalization
from keras.models import Sequential
import keras.optimizers as o
import keras

2024-10-31 09:27:08.132482: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-31 09:27:11.765376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 09:27:16.665950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 1:
    try:
        tf.config.set_visible_devices(gpus[1], 'GPU')
    except RuntimeError as e:
        print(e)
        
model = Sequential()

if len(X_train.shape) == 3:
    model.add(l.Input(shape=(X_train.shape[1],X_train.shape[2],1)))
    model.add(Conv2D(64,[1,48], 1, input_shape=(X_train.shape[1] ,X_train.shape[2],1), padding='same'))
    model.add(l.LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D((1,2),  padding='same'))

model.add(Conv2D(64,[1,8], 1, padding='same'))
model.add(l.LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D((1,2), padding='same'))

model.add(Conv2D(96,[1,6], 1, padding='same'))
model.add(l.LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D((1,2), padding='same'))

model.add(Conv2D(128,[1,5], 1, padding='same'))
model.add(l.LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D((1,2), padding='same'))

model.add(Conv2D(96,[1,6], 1, padding='same'))
model.add(l.LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D((1,2), padding='same'))

model.add(Flatten())

model.add(Dense(1024))
model.add(Dropout(0.5))
model.add(l.LeakyReLU())


model.add(Dense(512))
model.add(Dropout(0.6))
model.add(l.LeakyReLU())

model.add(Dense(Y_train.shape[1], activation='softmax'))

lr_schedule = o.schedules.ExponentialDecay(
    initial_learning_rate=5e-4,
    decay_steps=600,
    decay_rate=0.75)

opt = o.AdamW(learning_rate=lr_schedule, decay = 1e-6)
# opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["mse", "acc"])

model.summary()

callback = keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=True, patience=6)

/nfs/hpc/share/albousah/HTM/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-10-31 09:27:26.283608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 980 MB memory:  -> device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:61:00.0, compute capability: 9.0
/nfs/hpc/share/albousah/HTM/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 1, 137, 64)     │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 1, 137, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 137, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 1, 69, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 1, 69, 64)      │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 1, 69, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1, 69, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 35, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 35, 96)      │        36,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 1, 35, 96)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 35, 96)      │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 18, 96)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1, 18, 128)     │        61,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 1, 18, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1, 18, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 1, 9, 96)       │        73,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 1, 9, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 1, 9, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 5, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       492,544 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,243,359 (4.74 MB)

 Trainable params: 1,242,463 (4.74 MB)

 Non-trainable params: 896 (3.50 KB)

In [23]:
for i in range(3):
    M = model.fit(X_train,Y_train, batch_size=64, epochs=7, shuffle=True, verbose='auto', validation_split=0.01)#, callbacks=[callback])

Epoch 1/7


I0000 00:00:1730392049.814595 3136473 service.cc:145] XLA service 0x147f90008d30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730392049.818461 3136473 service.cc:153]   StreamExecutor device (0): NVIDIA H100 80GB HBM3, Compute Capability 9.0
2024-10-31 09:27:30.266032: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-31 09:27:31.476316: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8905
I0000 00:00:1730392055.842447 3140227 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_3814', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1730392057.230872 3140225 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2949', 76 bytes spill stores, 76 bytes spill loads



 71/961 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.2272 - loss: 2.9007 - mse: 0.0291

I0000 00:00:1730392079.703142 3136473 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


953/961 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.7213 - loss: 0.9415 - mse: 0.0117

I0000 00:00:1730392083.668810 3143407 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2903', 4 bytes spill stores, 4 bytes spill loads



961/961 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - acc: 0.7227 - loss: 0.9365 - mse: 0.0117 - val_acc: 1.0000 - val_loss: 0.0048 - val_mse: 7.9553e-06
Epoch 2/7
961/961 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9787 - loss: 0.0663 - mse: 0.0010 - val_acc: 1.0000 - val_loss: 0.0151 - val_mse: 5.8013e-05
Epoch 3/7
961/961 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9918 - loss: 0.0265 - mse: 4.1637e-04 - val_acc: 1.0000 - val_loss: 0.0015 - val_mse: 2.0860e-06
Epoch 4/7
961/961 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9934 - loss: 0.0205 - mse: 3.2734e-04 - val_acc: 0.9968 - val_loss: 0.0074 - val_mse: 1.0159e-04
Epoch 5/7
961/961 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9965 - loss: 0.0131 - mse: 1.8958e-04 - val_acc: 1.0000 - val_loss: 1.4928e-04 - val_mse: 4.2554e-08
Epoch 6/7
961/961 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9970 - loss: 0.0098 - mse: 1.4511e-04 - val_acc: 1.0000 - val_loss: 4.6611e-05 - val_mse: 3.9589e-09
Epoch 7/7
961/961 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9981 -

In [179]:
#################################################################
#################################################################

In [2]:
# Download the dataset from 
# Change Directory to where the data is

# For user to change 
Your_Dir = "..."

X_test = np.load(f"{Your_Dir}/X_test.npy")
Y_test = np.load(f"{Your_Dir}/Y_test.npy")

#################################################################

# Samples the first 137 components 
X_test = Preamble(X_test)

#################################################################

# Select a Technique [Mbed, TPD , TP] and comment out the others 

# X_test = Mbed(X_test)

X_test = TPD(X_test)

# X_test = TP(X_test)

#################################################################

Outputs = model.evaluate(X_test,Y_test)
print(f"Testing Loss = {Outputs[0]}\nTesting Accuracy = {Outputs[1]}\nTesting MSE = {Outputs[2]}")